In [1]:
import numpy as np # 今回は明示的には使わない
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb # LightGBM でモデルを作成する
from sklearn.metrics import accuracy_score, roc_auc_score #精度評価に使用

In [2]:
# 自作のモジュールを読み込み
# 自作のプロット用モジュールを読み込む
import sys
sys.path.append("../input/modeling-and-evaluation/")
import down_sampling as my_downsampling

In [3]:
# 各種定数の定義
# 乱数シード
RANDOM_SEED = 42
# target となる特徴量
TARGET_COULMN_NAME = "Exited"
# 削除する特徴量のリスト
DROP_COLUMNS = ["id", "CustomerId", "Surname"]
# one-hot encoding する特徴量のリスト
ONE_HOT_ENCODING_COLUMNS = ["Geography", "Gender"]

In [4]:
# 学習データの読み込み
train_df =  pd.read_csv("../input/tdse-inclass-competition-0/train.csv")

In [5]:
# down sampling
train_df = my_downsampling.downsampling(
    train_df,
    TARGET_COULMN_NAME,
    majority_query="Exited == 0",
    random_seed=RANDOM_SEED,
)

Exited
0    130113
1     34921
Name: count, dtype: int64


In [6]:
# 特徴量。Surname, id, CustomerIDは削除
X_train_df = train_df.drop(
    [TARGET_COULMN_NAME] + DROP_COLUMNS,
    axis=1,
) 
# one-hot encoding
X_train_df = pd.get_dummies(
    X_train_df,
    columns=ONE_HOT_ENCODING_COLUMNS,
)
# 目的変数
y_train_df = train_df[TARGET_COULMN_NAME]

# テストデータの読み込み
test_df =  pd.read_csv("../input/tdse-inclass-competition-0/test.csv")
# 学習データに合わせて特徴量を加工
X_test_df = test_df.drop(
    DROP_COLUMNS,
    axis=1,
)
# one-hot encoding
X_test_df = pd.get_dummies(
    X_test_df,
    columns=ONE_HOT_ENCODING_COLUMNS,
)

In [7]:
# LightGBM用のデータセット作成
train_data = lgb.Dataset(X_train_df, label=y_train_df)

In [8]:
# モデルのパラメータ設定
params = {
    "objective": "binary", # 2値分類
    "metric": "auc", # 評価指標は AUC under ROC curve
    "boosting_type": "gbdt", # 勾配ブースティング
    "learning_rate": 0.05, # 学習率
    "num_leaves": 31, # ノード数
    "feature_fraction": 0.9, # 各ツリーを作成する際の特徴量の使用割合
    "bagging_fraction": 0.8, # 各ツリーを作成する際のデータの使用割合
    "bagging_freq": 5, # バギング (データサンプリング) を行う頻度。5 なら 5回に1回バギングを行う
    "verbose": 0, # 学習中のログ出力の詳細度。0 なら何も出力しない
    "seed": RANDOM_SEED, # モデル全体の乱数シード
    "random_state": RANDOM_SEED, # モデル全体の乱数シード
    "bagging_seed": RANDOM_SEED, # バギングの乱数シード
    "feature_fraction_seed": RANDOM_SEED, # 特徴量の使用割合の乱数シード
}

# モデルのトレーニング
model = lgb.train(
    params,
    train_data,
)

[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42


In [9]:
# テストデータに対する予測
y_pred_np = model.predict(X_test_df)

In [10]:
y_pred_np

array([0.07631751, 0.93260914, 0.09364989, ..., 0.07903164, 0.40518747,
       0.47077364])

In [11]:
# 提出用ファイルの作成
submission_df = pd.DataFrame({
    "id": test_df["id"],
    "Exited": y_pred_np,
})

In [12]:
submission_df.head()

,id,Exited
0,165034,0.076318
1,165035,0.932609
2,165036,0.093650
3,165037,0.568738
4,165038,0.674822


In [13]:
submission_df.to_csv("submission.csv", index=False)